In [2]:
import cv2 as cv
import os

# Eğitilmiş modeli yükle
recognizer = cv.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')

# Yüz tespiti için cascade dosyasını yükle
cascadePath = cv.data.haarcascades + "haarcascade_frontalface_default.xml"
faceCascade = cv.CascadeClassifier(cascadePath)

font = cv.FONT_HERSHEY_SIMPLEX

# ID'lere karşılık gelen isimleri burada tanımla
names = ['Bilinmiyor', 'Eren', 'Enes','Zehra']
# Kamera bağlantısını başlat
cam = cv.VideoCapture(0)
cam.set(3, 640) # Genişlik
cam.set(4, 480) # Yükseklik

# Minimum pencere boyutu
minW = 0.1 * cam.get(3)
minH = 0.1 * cam.get(4)

while True:
    ret, img = cam.read()
    if not ret:
        print("Kamera okunamadı.")
        break
        
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(int(minW), int(minH)),
    )

    for (x, y, w, h) in faces:
        cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Yüzü tanı
        id, confidence = recognizer.predict(gray[y:y+h, x:x+w])

        # Güvenilirlik skorunu kontrol et (0 ise mükemmel eşleşme)
        # LBPH için genelde 100'den küçük değerler kabul edilebilir
        if confidence < 100:
            # ID'ye karşılık gelen ismi al
            id_name = names[id]
            confidence_text = f"  {round(100 - confidence)}%"
        else:
            id_name = names[0] # Bilinmiyor
            confidence_text = f"  {round(100 - confidence)}%"

        cv.putText(
            img,
            str(id_name),
            (x + 5, y - 5),
            font,
            1,
            (255, 255, 255),
            2
        )
        cv.putText(
            img,
            str(confidence_text),
            (x + 5, y + h - 5),
            font,
            1,
            (255, 255, 0),
            1
        )

    cv.imshow('kamera', img)

    k = cv.waitKey(10) & 0xff # Esc ile çık
    if k == 27:
        break

print("\n Programdan çıkılıyor.")
cam.release()
cv.destroyAllWindows()


 Programdan çıkılıyor.
